# Embed and Upload Blog Content to Azure AI Search (OpenAI >= 1.0.0)

In [ ]:
import json
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from openai import AzureOpenAI

# Load environment variables
load_dotenv()

# Azure OpenAI config
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

# Azure Cognitive Search config
search_client = SearchClient(
    endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    index_name=os.getenv("AZURE_SEARCH_INDEX"),
    credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
)

In [ ]:
# Load blog content JSON
with open("blog_documents.json", "r", encoding="utf-8") as f:
    documents = json.load(f)

In [ ]:
uploaded = 0
for doc in documents:
    try:
        print(f"Embedding doc {doc['id']} with model: {embedding_model}")
        response = client.embeddings.create(
            input=doc["content"],
            model=embedding_model
        )
        embedding = response.data[0].embedding

        enriched_doc = {
            "id": doc["id"],
            "content": doc["content"],
            "category": doc.get("category", "general"),
            "source": doc.get("source", "unknown"),
            "contentVector": embedding
        }

        search_client.upload_documents([enriched_doc])
        uploaded += 1

    except Exception as e:
        print(f"❌ Failed for doc {doc['id']}: {e}")

print(f"✅ Uploaded {uploaded} documents to Azure AI Search.")
